<div class="page" title="Page 3">
<div class="section" style="background-color: rgb(100.000000%, 100.000000%, 100.000000%);">
<div class="layoutArea">
<div class="column">
<p><strong><span style="font-size: 12.000000pt; font-family: 'CMR12';">Question 2:</span></strong></p>
<p><strong><span style="font-size: 12.000000pt; font-family: 'CMR12';">PART 1:&nbsp;</span></strong></p>
<p><span style="font-size: 12.000000pt; font-family: 'CMR12';">Use the MATLAB editor to open the scripts </span><span style="font-size: 12.000000pt; font-family: 'CMTT12';">InterceptionModel.m</span><span style="font-size: 12.000000pt; font-family: 'CMR12';">, and </span><span style="font-size: 12.000000pt; font-family: 'CMTT12';">SlugModel.m</span><span style="font-size: 12.000000pt; font-family: 'CMR12';">. These two functions compute the value of the cost function for given parameter values. Optimize the parameters of both models using your DE algorithm. A population of </span><span style="font-size: 12.000000pt; font-family: 'CMMI12';">N </span><span style="font-size: 12.000000pt; font-family: 'CMR12';">= 50 individuals with 250 generations should be more than sufficient to find the optimum parameter values. </span></p>
</div>
</div>
</div>
</div>

In [5]:
import numpy as np

def slug_model(pars):
    """
    Run the slug model and calculate the sum of squared residuals (SSR).

    Parameters:
    pars (list or array-like): List containing two parameters [S, T].

    Returns:
    float: Sum of squared residuals (SSR).
    """
    # Define the time of the observations
    t = np.array([5.0, 10.0, 20.0, 30.0, 40.0, 50.0])

    # Define the distance from injection
    d = 10

    # Define the amount of injected water
    Q = 50

    # Extract the storage (S) and transmissivity (T) from parameters
    S, T = pars

    # Predict "h" using the slug model equation
    h_pred = (Q / (4 * np.pi * T * t)) * np.exp((-d ** 2 * S) / (4 * T * t))

    
    # Define the measured head
    h_obs = np.array([0.55, 0.47, 0.30, 0.22, 0.17, 0.14])

    # Calculate residuals
    res = h_pred - h_obs

    # Calculate sum of squared residuals (SSR)
    SSR = np.dot(res, res)

    return SSR

# interceptmodel 
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
import scipy.io

# Load measurement data from a .mat file
data = scipy.io.loadmat('/dfs6/pub/gshalgum/CEE290AHW/measurement.mat')

# Extract data
obsTime = data['obsTime'].flatten()
obsPotEvap = data['obsPotEvap']
obsPrecip = data['obsPrecip']
obsStorage = data['obsStorage'].flatten()

# Interception Model function
def interceptionModel(t, S, P, E0, a, b, c, d):
    # Interpolators
    P_interp = interp1d(P[:, 0], P[:, 1], bounds_error=False, fill_value="extrapolate")
    E0_interp = interp1d(E0[:, 0], E0[:, 1], bounds_error=False, fill_value="extrapolate")

    # Interpolated values
    P_int = P_interp(t)
    E0_int = E0_interp(t)

    # Model calculations
    I = a * P_int
    D = b * np.maximum(S - c, 0)
    E = np.nan_to_num(d * E0_int * (S / c), nan=0.0) if c != 0 else 0

    # Change in storage
    dSdt = I - D - E
    return dSdt

# Residuals function for the Interception Model
def residuals(params, obsTime, obsPrecip, obsPotEvap, obsStorage):
    # Define a wrapper for solve_ivp
    def model(t, S):
        return interceptionModel(t, S, obsPrecip, obsPotEvap, *params)

    # Solve the differential equations for the full observation period
    sol = solve_ivp(model, [obsTime[0], obsTime[-1]], [obsStorage[0]], t_eval=obsTime, vectorized=True)
    simulatedStorage = sol.y.flatten()
    return obsStorage - simulatedStorage

# Objective function to minimize: sum of squared residuals
def interception_objective(params):
    residuals_vector = residuals(params, obsTime, obsPrecip, obsPotEvap, obsStorage)
    return np.dot(residuals_vector, residuals_vector)


/opt/apps/python/3.10.2/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
def slug_objective(params):
    return slug_model(params)

from scipy.optimize import differential_evolution

# Define bounds for Slug Model: assuming S and T have some typical ranges
slug_bounds = [(0.001, 0.01), (1, 100)]  


# Perform DE optimization for Slug Model
result_slug = differential_evolution(slug_objective, slug_bounds, strategy='best1bin', maxiter=250, popsize=50)
print("Optimized parameters for Slug Model:", result_slug.x)
print("Minimum SSR for Slug Model:", result_slug.fun)



Optimized parameters for Slug Model: [0.01       1.11397651]
Minimum SSR for Slug Model: 0.06920637142242358


In [7]:
# Intercept model 
import numpy as np
from scipy.optimize import differential_evolution
import scipy.io
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

# Load measurement data
data = scipy.io.loadmat('/dfs6/pub/gshalgum/CEE290AHW/measurement.mat')
obsTime = data['obsTime'].flatten()
obsPotEvap = data['obsPotEvap']
obsPrecip = data['obsPrecip']
obsStorage = data['obsStorage'].flatten()

def interceptionModel(t, S, P, E0, a, b, c, d):
    P_interp = interp1d(P[:, 0], P[:, 1], bounds_error=False, fill_value="extrapolate")
    E0_interp = interp1d(E0[:, 0], E0[:, 1], bounds_error=False, fill_value="extrapolate")
    P_int = P_interp(t)
    E0_int = E0_interp(t)
    I = a * P_int
    D = b * np.maximum(S - c, 0)
    E = np.nan_to_num(d * E0_int * (S / c), nan=0.0) if c != 0 else 0
    dSdt = I - D - E
    return dSdt

def residuals(params, obsTime, obsPrecip, obsPotEvap, obsStorage):
    def model(t, S):
        return interceptionModel(t, S, obsPrecip, obsPotEvap, *params)
    sol = solve_ivp(model, [obsTime[0], obsTime[-1]], [obsStorage[0]], t_eval=obsTime, vectorized=True)
    simulatedStorage = sol.y.flatten()
    return obsStorage - simulatedStorage

def interception_objective(params):
    residuals_vector = residuals(params, obsTime, obsPrecip, obsPotEvap, obsStorage)
    return np.dot(residuals_vector, residuals_vector)

# Define bounds for each parameter
interception_bounds = [(0, 1), (0, 1), (0.1, 10), (0, 1)]  # Update these if necessary

# Perform DE optimization
result_interception = differential_evolution(
    interception_objective,
    bounds=interception_bounds,
    strategy='best1bin',
    maxiter=250,
    popsize=50,
    disp=False  # DO NOT Displays convergence messages
)

print("Optimized parameters: a, b, c, d =", result_interception.x)
print("Minimum SSR:", result_interception.fun)



Optimized parameters: a, b, c, d = [0.50585366 0.88349476 3.30634715 0.94027394]
Minimum SSR: 0.5411053811954832
